In [ ]:
D = load_all_nonmfc()
X, Y = load_all_fv(3, 3)
svms = ['FV', 'eng', 'hcdf', 'brightness']
rf = ['zerocross', 'roughness']
def tail(it):
    next(it)
    return it
categories, arrs = zip(*sorted(list((k, v) for k, v in D.items() if k in set(svms + rf))))
ends = [0] + list(itertools.accumulate(a.shape[1] for a in arrs))
ends = zip(ends, ends[1:])
excerpts = dict(zip(categories, ends))
X = np.concatenate(arrs, axis=1)

In [ ]:
class SVCSub(SVC):
    def __init__(self, name):
        super(SVCSub, self).__init__(probability=True)
        self.name = name
    def fit(self, X, Y):
        super(SVCSub, self).fit(X[:,slice(*excerpts[self.name])], Y)
        return self
    def predict_proba(self, X):
        return super(SVCSub, self).predict_proba(X[:,slice(*excerpts[self.name])])
class RFSub(RandomForestClassifier):
    def __init__(self, name, md, ne):
        if not md: md = 10
        if not ne: ne = 50
        super(RFSub, self).__init__(max_depth=md, n_estimators=ne)
        self.name = name
    def fit(self, X, Y):
        super(RFSub, self).fit(X[:,slice(*excerpts[self.name])], Y)
        return self
    def predict_proba(self, X):
        return super(RFSub, self).predict_proba(X[:,slice(*excerpts[self.name])])
def make_classifier(name):
    if name == 'zerocross': return RFSub(name, 10, 50)
    if name == 'roughness': return RFSub(name, 20, 200)
    return SVCSub(name)